In [1]:
from tensorflow import keras
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow.keras.layers as layers
import scipy as sc

In [2]:
img_size=(240, 320)
img_channels = 3
batch_size=32

In [3]:
model_name = 'simple_covnet_model.tf'

In [4]:
train_size = 1000
validation_size = 500
test_size = 500

# Load the DataSet

Dataset citation:
- From Semi-Supervised to Transfer Counting of Crowds
C. C. Loy, S. Gong, and T. Xiang
in Proceedings of IEEE International Conference on Computer Vision, pp. 2256-2263, 2013 (ICCV)
- Cumulative Attribute Space for Age and Crowd Density Estimation
K. Chen, S. Gong, T. Xiang, and C. C. Loy
in Proceedings of IEEE Conference on Computer Vision and Pattern Recognition, pp. 2467-2474, 2013 (CVPR, Oral)
- Crowd Counting and Profiling: Methodology and Evaluation
C. C. Loy, K. Chen, S. Gong, T. Xiang
in S. Ali, K. Nishino, D. Manocha, and M. Shah (Eds.), Modeling, Simulation and Visual Analysis of Crowds, Springer, vol. 11, pp. 347-382, 2013
- Feature Mining for Localised Crowd Counting
K. Chen, C. C. Loy, S. Gong, and T. Xiang
British Machine Vision Conference, 2012 (BMVC)

In [5]:
dataset_path = '/Users/olove/Library/CloudStorage/OneDrive-Personal/AI datasets/CrowdCounter'

In [6]:
labels_df = pd.read_csv(dataset_path + '/labels.csv')
labels_df['image_name'] = labels_df['id'].map('seq_{:06d}.jpg'.format)
labels_df.drop("id", axis=1,inplace=True)
display(labels_df)

,count,image_name
0,35,seq_000001.jpg
1,41,seq_000002.jpg
2,41,seq_000003.jpg
3,44,seq_000004.jpg
4,41,seq_000005.jpg
...,...,...
1995,27,seq_001996.jpg
1996,27,seq_001997.jpg
1997,25,seq_001998.jpg
1998,26,seq_001999.jpg


In [7]:
labels_df = labels_df.sample(frac=1).reset_index(drop=True)
display(labels_df)

,count,image_name
0,32,seq_000355.jpg
1,40,seq_000741.jpg
2,31,seq_001697.jpg
3,27,seq_000512.jpg
4,42,seq_001715.jpg
...,...,...
1995,24,seq_000606.jpg
1996,30,seq_001966.jpg
1997,24,seq_001155.jpg
1998,43,seq_001652.jpg


In [8]:
if (train_size+validation_size+test_size) != len(labels_df):
    print('Dataset size is different from specified class sizes')
    exit(1)

training_df = labels_df[:train_size]
validation_df = labels_df[train_size:train_size+validation_size].reset_index(drop=True)
test_df = labels_df[train_size+validation_size:].reset_index(drop=True)

In [9]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
    rescale=1./255,
    
)

x_col_name = 'image_name'
y_col_name = 'count'

train_generator = datagen.flow_from_dataframe(
    training_df, 
    dataset_path + '/frames/frames/',
    x_col=x_col_name,
    y_col=y_col_name,
    class_mode='raw',
    target_size=img_size,
    batch_size=batch_size,
)

validation_generator = datagen.flow_from_dataframe(
    validation_df,
    dataset_path + '/frames/frames/',
    x_col=x_col_name,
    y_col=y_col_name,
    class_mode='raw',
    target_size=img_size,
    batch_size=batch_size,
)

test_generator = datagen.flow_from_dataframe(
    test_df,
    dataset_path + '/frames/frames/',
    x_col=x_col_name,
    y_col=y_col_name,
    class_mode='raw',
    target_size=img_size,
    batch_size=batch_size,
)

Found 1000 validated image filenames.
Found 500 validated image filenames.
Found 500 validated image filenames.


# Define the model

In [10]:
inputs = keras.Input(shape= img_size + (img_channels,))

## Custom Simple Covnet

Downsizing using strides instead of MaxPolling in order to conserve location data

In [11]:
#x = layers.Rescaling(1./255)(inputs)
x = layers.Conv2D(filters=32, kernel_size=3, strides=2, activation="relu")(inputs)
x = layers.Conv2D(filters=64, kernel_size=3, strides=2, activation="relu")(x)
x = layers.Conv2D(filters=128, kernel_size=3, strides=2, activation="relu")(x)
x = layers.Flatten()(x)
# x = layers.Dropout(0.5)(x)

2023-12-09 17:52:50.928000: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1 Pro
2023-12-09 17:52:50.928023: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2023-12-09 17:52:50.928028: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2023-12-09 17:52:50.928060: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-12-09 17:52:50.928073: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


# Output Layer

In [12]:
outputs = layers.Dense(128, activation="relu")(x)
outputs = layers.Dropout(0.5)(outputs)
outputs = layers.Dense(1)(outputs)

In [13]:
model = keras.Model(inputs=inputs, outputs=outputs)

In [14]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 240, 320, 3)]     0         
                                                                 
 conv2d (Conv2D)             (None, 119, 159, 32)      896       
                                                                 
 conv2d_1 (Conv2D)           (None, 59, 79, 64)        18496     
                                                                 
 conv2d_2 (Conv2D)           (None, 29, 39, 128)       73856     
                                                                 
 flatten (Flatten)           (None, 144768)            0         
                                                                 
 dense (Dense)               (None, 128)               18530432  
                                                                 
 dropout (Dropout)           (None, 128)               0     

# Train model

In [15]:
model.compile(loss="mse", optimizer="adam", metrics=["mae"])
# TODO: Try mae vs accuracy. mae should be better since we are adjusting it to get closer to the actual value

In [19]:
callbacks_list = [
#    keras.callbacks.EarlyStopping(
#        monitor="val_loss", patience=4
#    ),
    keras.callbacks.ModelCheckpoint(
        filepath=model_name,
        monitor="val_loss",
        save_best_only=True
    ),
    keras.callbacks.TensorBoard()
]

In [17]:
Wsave = model.get_weights()

In [20]:
model.set_weights(Wsave)
history = model.fit(train_generator,
        epochs=60,
        callbacks = callbacks_list,
        validation_data=validation_generator,
)

Epoch 1/60
32/32 [==============================] - ETA: 0s - loss: 180.8535 - mae: 10.4068INFO:tensorflow:Assets written to: simple_covnet_model.tf/assets


INFO:tensorflow:Assets written to: simple_covnet_model.tf/assets


32/32 [==============================] - 5s 152ms/step - loss: 180.8535 - mae: 10.4068 - val_loss: 107.0750 - val_mae: 8.6869
Epoch 2/60
32/32 [==============================] - ETA: 0s - loss: 121.0344 - mae: 8.7333INFO:tensorflow:Assets written to: simple_covnet_model.tf/assets


INFO:tensorflow:Assets written to: simple_covnet_model.tf/assets


32/32 [==============================] - 4s 138ms/step - loss: 121.0344 - mae: 8.7333 - val_loss: 59.5206 - val_mae: 6.1519
Epoch 3/60
32/32 [==============================] - ETA: 0s - loss: 121.2174 - mae: 8.7683INFO:tensorflow:Assets written to: simple_covnet_model.tf/assets


...

32/32 [==============================] - 4s 132ms/step - loss: 32.1341 - mae: 4.4699 - val_loss: 11.6852 - val_mae: 2.6489
Epoch 57/60
32/32 [==============================] - 4s 113ms/step - loss: 14.7062 - mae: 3.0220 - val_loss: 12.2421 - val_mae: 2.8344
Epoch 58/60
32/32 [==============================] - ETA: 0s - loss: 11.3090 - mae: 2.6624INFO:tensorflow:Assets written to: simple_covnet_model.tf/assets


INFO:tensorflow:Assets written to: simple_covnet_model.tf/assets


32/32 [==============================] - 4s 131ms/step - loss: 11.3090 - mae: 2.6624 - val_loss: 10.3818 - val_mae: 2.5629
Epoch 59/60
32/32 [==============================] - ETA: 0s - loss: 9.9008 - mae: 2.4887INFO:tensorflow:Assets written to: simple_covnet_model.tf/assets


INFO:tensorflow:Assets written to: simple_covnet_model.tf/assets


32/32 [==============================] - 4s 132ms/step - loss: 9.9008 - mae: 2.4887 - val_loss: 9.9769 - val_mae: 2.5117
Epoch 60/60
32/32 [==============================] - 4s 115ms/step - loss: 10.3229 - mae: 2.5124 - val_loss: 12.5020 - val_mae: 2.9107
